# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from diffusers import AutoPipelineForText2Image
import torch

# Используем SDXL-Turbo — 4 шага, быстро, мало VRAM
pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)
pipe.to("cuda")

def generate_image(prompt, filename="illustration.png"):
    full_prompt = (
        f"Акварельная иллюстрация в стиле импрессионизма, русская народная сказка — {" ".join(prompt.split(maxsplit=70))}. "
        "Impressionist watercolor, dreamy light, Russian fairy tale"
    )
    image = pipe(prompt=full_prompt, num_inference_steps=4, guidance_scale=0.0).images[0]

    os.makedirs("static", exist_ok=True)
    image.save(f"static/{filename}")
    return f"static/{filename}"

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import export_to_video
import torch
from PIL import Image

pipe_svd = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt-1-1",
    torch_dtype=torch.float16,
    variant="fp16",
    token=os.environ["HF_TOKEN"],
)
pipe_svd.to("cuda")

def generate_video(image_path, prompt, output_path="video.mp4", fps=5, num_frames=14):
    print(f"🎥 Генерация видео: {prompt[:50]}...")

    image = Image.open(image_path).convert("RGB")
    image = image.resize((576, 576))  # Меньше = меньше VRAM

    frames = pipe_svd(
        image,
        num_frames=num_frames,           # 14 кадров ≈ 3 сек при 5 FPS
        decode_chunk_size=1,             # Меньше VRAM
        motion_bucket_id=80,
        noise_aug_strength=0.02,
        height=576,
        width=576
    ).frames[0]

    export_to_video(frames, output_path, fps=fps)
    print(f"✅ Видео сохранено: {output_path}")
    return output_path

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
import ollama

class Agent:
    def __init__(self, name, system_prompt, model="qwen:14b-chat-q4_K_M"):
        self.name = name
        self.system_prompt = system_prompt
        self.model = model

    def respond(self, prompt):
        response = ollama.generate(
            model=self.model,
            prompt=f"{self.system_prompt}\n\n{prompt}",
            options={
                'temperature': 0.6,
                'top_p': 0.95,
                'top_k': 30,
                'num_ctx': 8192
            }
        )
        return response['response'].strip()

In [ ]:
import random
import os

# === Настройки ===
args = type('', (), {})()
args.generate_video = False  # Поменяйте на True, если хотите видео

# === Тема ===
PLOT_TEMPLATES = [
    "Царевну похитил змей, и добрый молодец отправляется её спасать.",
    "Царь заболел — нужна живая вода за тридевять земель."
]
chosen_plot = random.choice(PLOT_TEMPLATES)
print(f"🎯 ТЕМА СКАЗКИ: {chosen_plot}")

# === Этапы Кемпбелла ===
CAMPBELL_STAGES = [
    "1. Обычный мир",
    "2. Призыв к приключению",
    "3. Отказ от призыва",
    "4. Встреча с наставником",
    "5. Переход порога",
    "6. Испытания, союзники, враги",
    "7. Подземелье: приближение к пещере",
    "8. Кульминация: испытание огнём",
    "9. Вознаграждение",
    "10. Возвращение с даром"
]

# === Агенты ===
narrator = Agent("Рассказчик", "Ты — волшебный рассказчик. Говори на русском, в стиле народной сказки.")
child = Agent("Ребёнок", "Ты — ребёнок 7 лет. Реагируй по-русски: задавай вопросы, проси изменить.")
editor = Agent("Редактор", "Ты — редактор. Проверяй логику сказки.")

# === Хранение ===
history = []
full_tale = []
video_files = []

# === Цикл ===
for stage in CAMPBELL_STAGES:
    print(f"\n{'='*60}")
    print(f"{stage.upper()}")
    print(f"{'='*60}")

    stage_prompt = f"Расскажи этап сказки: {stage}. Тема: {chosen_plot}. Учти: {history[-2:] if history else 'начало'}"
    tale_fragment = narrator.respond(stage_prompt)
    history.append({"role": "Рассказчик", "content": tale_fragment})
    full_tale.append(f"**{stage}**\n{tale_fragment}")

    print(f"💬 Рассказчик: {tale_fragment}")

    child_resp = child.respond(f"Ты услышал: {tale_fragment}...")
    print(f"👶 Ребёнок: {child_resp}")

    editor_resp = editor.respond(f"Проверь логику: {tale_fragment}")
    print(f"📝 Редактор: {editor_resp}")

    # Картинка
    img_filename = f"stage_{CAMPBELL_STAGES.index(stage)+1}.png"
    img_path = generate_image(tale_fragment, img_filename)

    # Видео (опционально)
    if args.generate_video:
        video_filename = f"video_stage_{CAMPBELL_STAGES.index(stage)+1}.mp4"
        video_path = generate_video(img_path, tale_fragment, f"static/{video_filename}")
        if video_path:
            video_files.append(video_path)

# === Итог ===
print(f"\n📜 ПОЛНАЯ СКАЗКА:\n")
for part in full_tale:
    print(f"\n{part}\n{'-'*40}")

🎯 ТЕМА СКАЗКИ: Царевну похитил змей, и добрый молодец отправляется её спасать.

1. ОБЫЧНЫЙ МИР
